<div style="background-color: #e0d7fa; color: #000064; font-size: 26px; padding: 15px; border-radius: 5px;">
   Extraheren van enkele Key Performance Indicators  - KPIs
</div>

In [1]:
# Dependencies:
!pip install -r ./code/requirements.txt -q

<div style="background-color: #e0e7fa; color: #000064; font-size: 20px; padding: 15px; border-radius: 5px;">
  Enkele imports ...
</div>

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Ophalen van 'the electric hour', enkel de relevant kolommen
df_electric_hour = (pd.read_csv('data/electric_hour.csv', parse_dates=['Timestamp']).set_index('Timestamp')
        .drop(columns=['Validatiestatus', 'Power(W)', 'AC_Voltage', 'Temperature(℃)', 'Grid_frequency', 'daglicht(s)', 'solstice_distance(days)']))

In [4]:
df_electric_hour.head()

,Afname (Kwh),Injectie (Kwh),1h_Generated(kWh)
Timestamp,,,
2021-09-01 00:00:00,0.078,0.0,NaN
2021-09-01 01:00:00,0.094,0.0,NaN
2021-09-01 02:00:00,0.089,0.0,NaN
2021-09-01 03:00:00,0.077,0.0,NaN
2021-09-01 04:00:00,0.095,0.0,NaN


In [5]:
df_KPIs  = df_electric_hour.groupby(df_electric_hour.index.year).sum().drop([2021, 2025]) #suomeren per jaar en onvolledige jaren droppen
df_KPIs = df_KPIs.rename(columns= {'1h_Generated(kWh)' : 'Opgewekt (kWh)'})

zelfverbruik = df_KPIs['Opgewekt (kWh)']+df_KPIs['Injectie (Kwh)']
totale_verbuik = df_KPIs['Afname (Kwh)']+ zelfverbruik

# Hoeveel van mijn opwekking blijft er in huis 'plakken' oftwel, wel percentage van mijn opwek verbruik ikzelf?

df_KPIs['Zelfverbruik (%)'] = round(100*(zelfverbruik/df_KPIs['Opgewekt (kWh)']),1)

# Hoeveel van mijn totale verbruik (afname + zelfverbruik) is zelfverbruik? 

df_KPIs['Zelfvoorziening(%)'] =  round(100*zelfverbruik/totale_verbuik,1)

#heb ik, geven huidige verbruiken, genoeg zonnepanelen om eventueel mijn volledige stroomverbruik te dekken? +ve marge = volfoende

df_KPIs['marge PV capaciteit?'] = df_KPIs['Opgewekt (kWh)']-totale_verbuik

df_KPIs

,Afname (Kwh),Injectie (Kwh),Opgewekt (kWh),Zelfverbruik (%),Zelfvoorziening(%),marge PV capaciteit?
Timestamp,,,,,,
2022,1130.202,-2271.832,2778.800741,18.2,31.0,1141.630
2023,1239.151,-1976.910,2638.200000,25.1,34.8,737.759
2024,1324.545,-1675.477,2333.700000,28.2,33.2,350.932


<div style="background-color: #e0e7fa; color: #000064; font-size: 20px; padding: 15px; border-radius: 5px;">
  Even een natte-vinger simulatie...
</div>

Ik gebruik zo'n 3kWh aan aardgas dagelijks voor de douche en ander warmwaternoden. Stel dat we dit 1:1 vervangen is met een elektrische boiler - dit zou mijn totale verbruik met(3*365 ≈≈ 1100) kwh verhogen.

Nu kan ik niet gemakkelijk weten hoe dat het zelfverbruik/Zelfvoorziening zou beïnvloeden, maar de PV capaciteit marge is hiervan onafhankelijk. Heb ik dan nog zonnepanelen genoeg? 

In [6]:
df_KPIs  = df_electric_hour.groupby(df_electric_hour.index.year).sum().drop([2021, 2025]) #suomeren per jaar en onvolledige jaren droppen
df_KPIs = df_KPIs.rename(columns= {'1h_Generated(kWh)' : 'Opgewekt (kWh)'})
df_KPIs['Afname (Kwh)'] = df_KPIs['Afname (Kwh)']+1100
zelfverbruik = df_KPIs['Opgewekt (kWh)']+df_KPIs['Injectie (Kwh)']
totale_verbuik = df_KPIs['Afname (Kwh)']+ zelfverbruik

#heb ik, geven huidige verbruiken, genoeg zonnepanelen om eventueel mijn volledige stroomverbruik te dekken? +ve marge = volfoende

df_KPIs['marge PV capaciteit zonder aardgas ?'] = df_KPIs['Opgewekt (kWh)']-(totale_verbuik)

df_KPIs

,Afname (Kwh),Injectie (Kwh),Opgewekt (kWh),marge PV capaciteit zonder aardgas ?
Timestamp,,,,
2022,2230.202,-2271.832,2778.800741,41.630
2023,2339.151,-1976.910,2638.200000,-362.241
2024,2424.545,-1675.477,2333.700000,-749.068
